<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/McCabeThiele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.3 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
from scipy.interpolate import Akima1DInterpolator
jax.config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots
from scipy.optimize import root

In [3]:
p=Props(["Benzene","Toluene"])

In [4]:
def bubbleP_ideal(x, T):
    Pi = x*p.Pvap(T)
    P = np.sum(Pi)
    return P, Pi/P

In [5]:
z=np.array([0.3, 0.7])
z

array([0.3, 0.7])

In [6]:
bubbleP_ideal(z, 373.15)

(Array(105789.27018675, dtype=float64),
 Array([0.51014734, 0.48985266], dtype=float64))

In [7]:
def bubbleT_ideal(x, P):

    def P_dev(T):
        return bubbleP_ideal(x,T)[0] - P

    T=root(P_dev, 300.).x[0]
    return T,x*p.Pvap(T)/P


In [8]:
bubbleT_ideal(z, 101325.)

(371.6543967389883, Array([0.51137952, 0.48862048], dtype=float64))

In [9]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

In [10]:
dewP_ideal(z, 373.15)

(Array(89901.78980933, dtype=float64),
 Array([0.14992484, 0.85007516], dtype=float64))

In [11]:

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [12]:
dewT_ideal(z, 89901.79)

(373.15000007098485, Array([0.14992484, 0.85007516], dtype=float64))

In [13]:
z1 = np.linspace(0.,1.,20)
zs=np.c_[z1, 1-z1]

In [14]:
x1s=[]
y1s=[]
Ps=[]
T=373.15
for z in zs:
    P, y =bubbleP_ideal(z, T)
    x1s.append(z[0])
    y1s.append(y[0])
    Ps.append(P)

fig=make_subplots()
fig.add_scatter(x=x1s, y=Ps, name='Bubble')
fig.add_scatter(x=y1s, y=Ps, name='Dew')
fig.update_layout(width=600, height=400, template='plotly_dark',
                  title=f'Benzene (1)/Toluene (2) T={T}')

In [15]:
x1s=[]
y1s=[]
Ts=[]
P=101325.
for z in zs:
    T, y =bubbleT_ideal(z, P)
    x1s.append(z[0])
    y1s.append(y[0])
    Ts.append(T)

fig=make_subplots()
fig.add_scatter(x=x1s, y=Ts, name='Bubble')
fig.add_scatter(x=y1s, y=Ts, name='Dew')
fig.update_layout(width=600, height=400, template='plotly_dark',
                  title=f'Benzene (1)/Toluene (2) P={P}')

In [16]:
fig3=make_subplots()
fig3.add_scatter(x=x1s, y=y1s)
fig3.add_scatter(x=[0.,1.],y=[0.,1.], mode='lines', line_color='rgb(50,50,50)')
fig3.update_layout(width=400,height=400,template='plotly_dark', showlegend=False)

In [17]:
p1, p2= p.Pvap(373.15)
alpha = p1/p2

In [18]:
x1plot = np.linspace(0,1,50)
y1plot = alpha*x1plot/((1-x1plot) + alpha*x1plot)

In [19]:
fig4=make_subplots()
fig4.add_scatter(x=x1s, y=y1s, mode='markers')
fig4.add_scatter(x=x1plot, y=y1plot, mode='lines')
fig4.add_scatter(x=[0.,1.],y=[0.,1.], mode='lines', line_color='rgb(50,50,50)')
fig4.update_layout(width=400,height=400,template='plotly_dark', showlegend=False)

In [28]:
y1_spline = Akima1DInterpolator(x1s, y1s)
x1_spline = Akima1DInterpolator(y1s, x1s)

In [52]:
F=1.
zF = 0.4
q = 0.4  #liquid fraction of feed
rD = 0.95 #Fraction of more volatile component recovered in distillate
xD = 0.98 #Distillate composition
D =  (rD * zF *F)/xD
B = F - D
xB=(1-rD)*zF*F/B
R = 6
L = R * D
V = L + D

Lb = L + q*F
Vb = V - (1-q)*F
Rb = Vb/B

xs=[xD]
ys=[xD]
y=xD
for i in range(1000):
    xeq = x1_spline(y)
    xs.append(xeq)
    ys.append(y)
    if xeq<xB:
        break
    if xeq > x_int:
        y=R/(R+1)*xeq + xD/(R+1)
    else:
        y=(Rb+1)*xeq/Rb - xB/Rb
    xs.append(xeq)
    ys.append(y)
xs=np.r_[xs]
ys=np.r_[ys]

In [64]:
fig5=make_subplots()
fig5.add_scatter(x=[xD, 0], y=[xD, xD/(R+1)], mode='lines', line_color='rgb(0,100,0)')  #rectifying
fig5.add_scatter(x=[xB, (Vb + B*xB)/Lb], y=[xB, 1], mode='lines', line_color='rgb(0,100,0)') #stripping

x_int = (xD*D/V + xB*B/Vb)/(Lb/Vb - L/V)
y_int = L/V * x_int + xD*D/V
fig5.add_scatter(x=[zF, x_int], y=[zF, y_int], mode='lines', line_color='rgb(0,100,0)')  #q-line

fig5.add_scatter(x=xs, y=ys, mode='lines', line_color='rgb(200,200,200)')  #q-line
fig5.add_scatter(x=x1plot, y=y1_spline(x1plot), mode='lines')
fig5.add_scatter(x=[0.,1.],y=[0.,1.], mode='lines', line_color='rgb(50,50,50)')
for i,(x,y) in enumerate(zip(xs[1::2],ys[1::2]), start=1):
    fig5.add_annotation(x=x,y=y, text=f'{i}', showarrow=False, yshift=10)


fig5.update_layout(width=400,height=400,template='plotly_dark', showlegend=False)

1 0.9497081211033829 0.98
2 0.8893484025232458 0.9540355323743281
3 0.7829467313874243 0.9022986307342106
4 0.6295921520717659 0.8110971983320779
5 0.4605679477765914 0.6796504160615136
6 0.31956064875484314 0.5347725266656497
7 0.21794682703712726 0.40263805669543473
8 0.13481731739134473 0.2715993831289595
9 0.07653118499877724 0.16439762165524763
10 0.039825831250489906 0.08923326559687876
11 0.018199494343828237 0.04189894840796767
